In [4]:
from typing import Optional, Union, Callable, List
 
import numpy as np
import pyspark
import shapely
import pyspark.sql.functions as F
import pandas as pd
import geopandas as gpd
from pyspark.sql.types import BooleanType
from shapely.geometry import Polygon, Point
from geopandas import GeoDataFrame
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("URBANICITY").getOrCreate()

In [5]:
%run ./function_tools.ipynb

In [6]:
def haversine(coord1: float, coord2: float) -> float:
    if coord1 is None or coord2 is None:
        return None
    # Coordinates in decimal degrees (e.g. 43.60, -79.49)
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    R = 6371000  # radius of Earth in meters
    phi_1 = np.radians(lat1)
    phi_2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi_1) * np.cos(phi_2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1 - a))
    meters = R * c  # output distance in meters
#     km = meters / 1000.0  # output distance in kilometers
    meters = round(meters)
#     km = round(km, 3)
    #print(f"Distance: {meters} m")
    #print(f"Distance: {km} km")
    return meters
haversine_sdf = F.pandas_udf(function_vectorizer(haversine), "double")

In [7]:
def haversine_distance(longit_a: float, latit_a: float,
        longit_b: float, latit_b: float) -> float:
    """This function takes the geocoordinates of 2 points and returns the 
    haversine distance between them, measured in meters.
    
    Arguments:
        longit_a {float} -- The longitude of the first point in decimal
            coordinates.
        latit_a {float} -- The latitude of the first point in decimal
            coordinates.
        longit_b {float} -- The longitude of the second point in decimal
            coordinates.
        latit_b {float} -- The latitude of the second point in decimal
            coordinates.
    
    Returns:
        float -- The Haversine (large circle) distance between the two points.
    """
 
    # Transform to radians
    longit_a, latit_a, longit_b, latit_b = map(np.radians, [longit_a,  latit_a, longit_b, latit_b])
    dist_longit = longit_b - longit_a
    dist_latit = latit_b - latit_a
    # Calculate area
    area = np.sin(dist_latit/2)**2 + np.cos(latit_a) * np.cos(latit_b) * np.sin(dist_longit/2)**2
    # Calculate the central angle
    central_angle = 2 * np.arcsin(np.sqrt(area))
    #   central_angle = 2 * np.arctan2(np.sqrt(area), np.sqrt(1-area))
    radius = 6371000
    # Calculate Distance
    distance = central_angle * radius
    return abs(round(distance, 2))
  
haversine_distance_sdf = F.pandas_udf(function_vectorizer(haversine_distance), "double")

In [8]:
def filter_osm(sdf: pyspark.sql.dataframe.DataFrame,
               contains_any: Optional[Union[str, List[str]]] = None,
               contains_all: Optional[Union[str, List[str]]] = None,
               contains_neither: Optional[Union[str, List[str]]] = None,
               preserve: List[str] = ["osm_id", "latitude", "longitude"],
               minimal: bool = True) -> pyspark.sql.dataframe.DataFrame:
  """This function takes a Spark Data Frame with a transformed OSM data set
  (meaning in long tabular format, with the columns 'osm_id', 'latitude' and
  'longitude' present), applies some basic filtering, and returns the filtered
  set. For simplicity, the only filtering options are: (contains_any OR 
  contains_all) AND contains_neither.
  
  Args:
      sdf (pyspark.sql.dataframe.DataFrame): The starting transformed OSM data frame.
      contains_any (Optional[Union[str, List[str]]], optional): String or list of strings, specifying
        records with which OSM tag/values should be left in the filtered set. They need to contain ANY
        of the string values to satisfy the contidion. Can't be used together with 'contains_all'.
        Defaults to None.
      contains_all (Optional[Union[str, List[str]]], optional): String or list of strings, specifying
        records with which OSM tag/values should be left in the filtered set. They need to contain ALL
        of the string values to satisfy the contidion. Can't be used together with 'contains_any'.
        Defaults to None.
      contains_neither (Optional[Union[str, List[str]]], optional): String or list of strings, specifying
        records with which OSM tag/values should NOT be left in the filtered set. They MUST NOT contain
        ANY of the string values to satisfy the condition. Defaults to None.
      preserve (List[str], optional): Specify which columns should not be used for the term search for
        inclusion/exclusion. Defaults to ["osm_id", "latitude", "longitude"].
      minimal (bool, optional): If set to True, only the columns mentioned in 'preserve' will be included
        in the returned Data Frame. Defaults to True.
 
  Returns:
      pyspark.sql.dataframe.DataFrame: Contains only the records which satisfy the conditions of
        containing the string in 'contains_any' or 'contains_all' and, optionally, NOT containsing the
        strings in 'contains_neither'.
        
  TODO: Raise appropriate errors in place of 'print("Error")'.
  """
  
  if contains_any is not None and contains_all is not None:
    print("Only one of these must be set")
  
  # concat everything except for the columns to be preserved
  sdf_concat = sdf.withColumn('__concatenated', F.concat_ws(' ',*set(sdf.columns).difference(set(preserve))))
  
  # build the filter on contains 'either' or 'all'
  contains_filter = None
  if contains_any is not None:
    if isinstance(contains_any, str):
      contains_filter = F.col("__concatenated").contains(contains_any)
    elif isinstance(contains_any, list):
      contains_filter = F.col("__concatenated").contains(contains_any[0])
      for i in range(1, len(contains_any)):
        contains_filter = contains_filter | F.col("__concatenated").contains(contains_any[i])
    else:
      print("Error")
  elif contains_all is not None:
    if isinstance(contains_all, str):
      contains_filter = F.col("__concatenated").contains(contains_all)
    elif isinstance(contains_all, list):
      contains_filter = F.col("__concatenated").contains(contains_all[0])
      for i in range(1, len(contains_all)):
        contains_filter = contains_filter & F.col("__concatenated").contains(contains_all[i])
    else:
      print("Error")
  else:
    print("Error")
  
  # build the filter on not containing
  contains_not_filter = None
  if contains_neither is not None:
    if isinstance(contains_neither, str):
      contains_not_filter = ~F.col("__concatenated").contains(contains_neither)
    elif isinstance(contains_neither, list):
      contains_not_filter = F.col("__concatenated").contains(contains_neither[0])
      for i in range(1, len(contains_neither)):
        contains_not_filter = contains_not_filter | F.col("__concatenated").contains(contains_neither[i])
      contains_not_filter = ~contains_not_filter
    else:
      print("Error")
  
  sdf_filtered = sdf_concat.filter(contains_filter)
  if contains_not_filter is not None:
    sdf_filtered = sdf_filtered.filter(contains_not_filter)
    
  if minimal:
    return sdf_filtered.select(preserve)
  
  return sdf_filtered


In [9]:
def sdf_to_gdf(sdf: pyspark.sql.dataframe.DataFrame,
               longitude: str = "longitude",
               latitude: str = "latitude",
               crs: str = "epsg:4326"
              ) -> GeoDataFrame:
  
  pdf = sdf.toPandas()
  gdf = gpd.GeoDataFrame(pdf, geometry=gpd.points_from_xy(pdf[longitude], pdf[latitude]), crs=crs)
  return(gdf)

In [10]:
def geo_aggregate(base_gdf: GeoDataFrame,
                  add_gdf: GeoDataFrame,
                  base_id: str,
                  add_id: str,
                  aggfunc: Callable,
                  new_col_name: str
                 ) -> Optional[GeoDataFrame]:
  gjoin = gpd.sjoin(base_gdf[[base_id, "geometry"]], add_gdf[[add_id, "geometry"]], how="inner")
  if gjoin.shape[0]==0:
    return None
  agg = pd.pivot_table(gjoin, index=base_id, aggfunc={add_id: aggfunc}).reset_index()[[base_id, add_id]].rename(columns={add_id: new_col_name})
  res = base_gdf.merge(agg, how="left", on="CUSTOMER_ID")
  res[new_col_name] = res[new_col_name].fillna(0)
  return(res)


In [11]:
def add_buffer_meters(gdf: GeoDataFrame, geometry_col: str, buffer_col: str, output_geometry_col: str, buffer_factor: float = 1., crs_calc: str = "epsg:3857", crs_out: str = "epsg:4326") -> GeoDataFrame:
  gdf[output_geometry_col] = gdf[geometry_col].to_crs(crs_calc).buffer(gdf[buffer_col] * buffer_factor).to_crs(crs_out)
  return(gdf)

In [12]:
def add_raster_stats(gdf: GeoDataFrame, raster_file: str, col_prefix: str, stats: List[str], crs_out: str = "epsg:4326") -> GeoDataFrame:
  return(GeoDataFrame.from_features(rs.zonal_stats(gdf, raster_file, prefix=col_prefix, stats=stats, geojson_out=True), crs=crs_out))


In [13]:
def make_polygon_filter(polygon: shapely.geometry.Polygon) -> Callable:
  @F.pandas_udf("boolean")
  @function_vectorizer
  def point_in_polygon(longitude: float,
                       latitude: float
                       ) -> bool:
    return polygon.contains(Point(longitude, latitude))
  return point_in_polygon